## 파이썬으로 PDF 파일 원하는 대로 추출하기


샘플문서 : 네이버 2023 사업보고서 (https://www.navercorp.com/investment/irReport)

In [ ]:
## 필요 모듈 다운로드
!pip install --upgrade pymupdf


In [ ]:
import pymupdf  
from typing import List, Tuple, Dict

In [ ]:
# pdf 파일을 열어보기
pdf_path = "네이버_2023_사업보고서.pdf"

doc = pymupdf.open(pdf_path)
print(doc)

In [ ]:
## document는 각각의 페이지로 구성된 리스트

## 아래를 보면 페이지 수랑 일치함
print(len(doc))

## 첫페이지는 이렇게 접근 가능함
print(doc[0])  

In [ ]:
## 페이지의 정보를 dict 형태로 반환
doc[0].get_text("dict")

## 기본 페이지정보 (너비(width),높이(height) 너비, 블럭) 등이 있음
## 1개의 블럭은 1개의 사각형 영역을 의미


In [ ]:
## 기본적으로 1개의 페이지는 여러 블럭으로 나눠어져 있음

## 회사의 개요 페이지는 33개 블럭으로 되어있네
len(doc[5].get_text("dict")["blocks"])

In [ ]:
## 블럭정보에는 
# - number : 블럭번호, 0이면 페이지의 첫번째 블럭)
# - type : 블럭타입, 0이면 텍스트, 1이면 이미지
# - bbox : 블럭의 경계좌표
# - lines : 여러 줄로 이루어진 문단일 경우 여러 개의 줄이 포함된 리스트
# 블럭 1개를 자세히 보면 각각의 줄 (line)로 이루어져 있음

## 1개의 줄에 그 줄의 텍스트 정보가 나와있음
# - text : 실제로 우리가 관심이 있는 텍스트
# - font : 글꼴
# - size : 글자크기

doc[5].get_text("dict")["blocks"][0]

In [ ]:
## 그래서 우리는 pdf 문서안의 모든 페이지, 모든 블럭, 모든 줄을 돌면서
## 모든 텍스트와 폰트크기를 같이 구해서 리스트로 만들어줄 것임

## 일단 모든 텍스트를 텍스트 / 폰트크기 조합으로 추출해보기
TextBlock = Tuple[str, float]

def extract_texts_with_font_size(pdf_path: str) -> List[TextBlock]:
    doc = pymupdf.open(pdf_path)
    content = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            lines = block.get("lines", [])

            for line in lines:
                for span in line["spans"]:
                    text = span["text"]
                    size = span["size"]
                    content.append((text, size))
    
    return content
result = extract_texts_with_font_size(pdf_path)


In [ ]:
for i in result[1000:1100]:
  print(i)

In [ ]:
## 소제목이 되는 텍스트의 글자 크기를 확인해보기 => 16인걸 확인했음!

some_titles = ["1. 사업의 개요", "2. 주요 제품 및 서비스"]
title_font_sizes = []

for text, size in result:
        if text in some_titles:
            title_font_sizes.append((text, size))
print(title_font_sizes)

In [ ]:
## 특정 글자 크기 기준으로 텍스트들을 쪼개는 함수

def create_chunks_based_on_font_size(content: List[TextBlock], target_font_size: float) -> List[List[TextBlock]]:
    chunks = []  # 텍스트 블록의 청크를 저장할 리스트
    current_chunk = []  # 현재 청크를 저장할 임시 리스트

    for text_block in content:
        text, font_size = text_block

        if font_size == target_font_size:
            # 대상 글꼴 크기를 가진 텍스트 블록을 찾았습니다.
            # 이는 새로운 청크의 시작을 의미합니다.

            # 현재 청크에 텍스트 블록이 있으면 저장합니다.
            if current_chunk:
                chunks.append(current_chunk)
            
            # 이 텍스트 블록으로 새로운 청크를 시작합니다.
            current_chunk = [text_block]
        else:
            # 이 텍스트 블록은 대상 글꼴 크기를 가지고 있지 않습니다.
            # 현재 청크에 추가합니다.
            current_chunk.append(text_block)

    # 마지막 청크가 비어 있지 않으면 추가합니다.
    if current_chunk:
        chunks.append(current_chunk)

    return chunks


In [ ]:

# 테스트용 데이터
content = [
    ("제목 1", 12.0),
    ("내용 1 아래 내용입니다.", 10.0),
    ("제목 2", 12.0),
    ("제목 2 아래 내용입니다.", 10.0),
    ("제목 2 아래 추가 내용입니다.", 10.0),
    ("제목 3", 12.0),
    ("제목 3 아래 내용입니다.", 10.0)
]
target_font_size = 12.0
chunks = create_chunks_based_on_font_size(content, target_font_size)
for i, chunk in enumerate(chunks):
    print(f"\n 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

In [ ]:
## 그대로 사업보고서 데이터에 적용해보기 -> 44개의 덩어리들이 만들어짐

chunks = create_chunks_based_on_font_size(result, 16)
len(chunks)

In [ ]:

## 이 중에서 특정 제목 기준으로 필터링하기


def filter_chunks_by_text_list(chunks: List[List[TextBlock]], delimiter_texts: List[str]) -> List[List[TextBlock]]:
    filtered_chunks = []
    
    for chunk in chunks:
        # 청크의 첫 번째 텍스트가 구분자 텍스트 목록의 어느 하나와 일치하는지 확인
        if chunk and chunk[0][0] in delimiter_texts:
            filtered_chunks.append(chunk)
    
    return filtered_chunks

# 특정 텍스트 목록을 포함하는 청크를 필터링합니다.
delimiter_texts = ["1. 사업의 개요", "2. 주요 제품 및 서비스"]
filtered_chunks = filter_chunks_by_text_list(chunks, delimiter_texts)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

In [ ]:
## 주석처리된 특정 텍스트들은 다시한번 지워주기
## 보통 주석은 같은 글자 크기인 경우가 많음
##   전자공시시스템 dart.fss.or.kr (9.0)

def exclude_chunks_based_on_font_size(chunks: List[List[TextBlock]], exclude_font_size: float) -> List[List[TextBlock]]:
    """특정 글꼴 크기에 따라 청크를 제외합니다."""
    filtered_chunks = []
    
    for chunk in chunks:
        temp_chunk = []
        for text, size in chunk:
            # exclude_font_size와 일치하는 글꼴 크기를 가진 텍스트 블록을 건너뜁니다.
            if size == exclude_font_size:
                continue
            temp_chunk.append((text, size))
        
        # 임시 청크가 비어 있지 않으면 필터링된 청크에 추가합니다.
        if temp_chunk:
            filtered_chunks.append(temp_chunk)
    
    return filtered_chunks


filtered_chunks_2 = exclude_chunks_based_on_font_size(filtered_chunks,9)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks_2):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

In [ ]:
## 최종 결과 한번 인쇄해보기

concatenated_texts = []
    
for chunk in filtered_chunks_2:
    concatenated_text = "".join(text for text, _ in chunk)
    concatenated_texts.append(concatenated_text)

for text in concatenated_texts:
    print(text)    